In [1]:

import pandas as pd
import matplotlib as plt
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split


2022-12-06 20:04:51.782333: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Reading Dataset:
dataset = pd.read_csv("Resources/Prepped_Stroke_Data.csv")
# Top 5 records:
dataset.head()

,age,gender,nhiss,severity_level,mrs,systolic,distolic,glucose,paralysis,smoking,bmi,cholestrol,tos,risk
0,75,1,4,1,1,140,89,190,1,1,25,205,1,1
1,68,1,1,1,1,133,116,174,1,0,35,206,1,1
2,88,0,1,1,1,124,118,79,2,0,35,239,1,1
3,58,0,4,1,0,126,105,198,0,0,25,222,1,1
4,50,1,3,1,2,140,106,87,0,0,39,222,1,2


In [3]:
dataset.corr()

,age,gender,nhiss,severity_level,mrs,systolic,distolic,glucose,paralysis,smoking,bmi,cholestrol,tos,risk
age,1.000000,-0.033676,0.058495,0.033389,0.041504,0.439812,0.476497,-0.050585,0.089203,0.320757,0.015409,0.471120,-0.013581,0.122307
gender,-0.033676,1.000000,-0.171225,-0.147354,-0.098114,-0.078162,-0.073362,-0.044245,-0.041195,-0.056915,-0.022629,-0.086824,-0.095736,-0.092974
nhiss,0.058495,-0.171225,1.000000,0.930975,0.695623,0.369223,0.254689,0.437065,0.195058,0.101446,0.145126,0.223306,0.534138,0.557640
severity_level,0.033389,-0.147354,0.930975,1.000000,0.767575,0.388640,0.239997,0.518909,0.201266,0.086293,0.140277,0.187086,0.581724,0.607909
mrs,0.041504,-0.098114,0.695623,0.767575,1.000000,0.377404,0.218693,0.544069,0.190380,0.044584,0.167718,0.164646,0.521503,0.556399
systolic,0.439812,-0.078162,0.369223,0.388640,0.377404,1.000000,0.590822,0.299251,0.190716,0.226261,0.118996,0.531017,0.284736,0.412328
distolic,0.476497,-0.073362,0.254689,0.239997,0.218693,0.590822,1.000000,0.151600,0.179014,0.279731,0.111874,0.541236,0.184149,0.288969
glucose,-0.050585,-0.044245,0.437065,0.518909,0.544069,0.299251,0.151600,1.000000,0.164600,-0.047011,0.120565,0.104890,0.368605,0.502262
paralysis,0.089203,-0.041195,0.195058,0.201266,0.190380,0.190716,0.179014,0.164600,1.000000,0.059543,0.107851,0.157792,0.116854,0.258843
smoking,0.320757,-0.056915,0.101446,0.086293,0.044584,0.226261,0.279731,-0.047011,0.059543,1.000000,0.006805,0.280354,0.063322,0.177899


In [4]:
X = dataset.drop(["nhiss","severity_level","mrs","tos", "risk"],axis=1)
y = dataset["severity_level"]
X

,age,gender,systolic,distolic,glucose,paralysis,smoking,bmi,cholestrol
0,75,1,140,89,190,1,1,25,205
1,68,1,133,116,174,1,0,35,206
2,88,0,124,118,79,2,0,35,239
3,58,0,126,105,198,0,0,25,222
4,50,1,140,106,87,0,0,39,222
...,...,...,...,...,...,...,...,...,...
4545,67,0,180,83,198,1,2,39,219
4546,67,1,180,88,188,0,2,22,187
4547,65,0,126,111,227,0,3,23,233
4548,64,0,126,82,262,2,2,28,244


In [5]:
y_adjusted = pd.get_dummies(y.replace(1,"mild").replace(2,"mild_to_moderate").replace(3,"severe").replace(4,"very_severe"),drop_first=True)
y_adjusted

,mild_to_moderate,severe,very_severe
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
4545,1,0,0
4546,0,1,0
4547,1,0,0
4548,0,1,0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y_adjusted, random_state=98)

# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
nn_model2 = tf.keras.models.Sequential()
nn_model2.add(tf.keras.layers.Dense(units=9, activation="relu", input_dim=9))
nn_model2.add(tf.keras.layers.Dense(units=9, activation="relu"))
#nn_model2.add(tf.keras.layers.Dense(units=11, activation="relu"))

nn_model2.add(tf.keras.layers.Dense(units=3, activation="sigmoid"))
nn_model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn_model2.fit(X_train_scaled, y_train, epochs=400)

Epoch 1/400


2022-12-06 20:06:00.779183: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


107/107 [==============================] - 1s 3ms/step - loss: 0.6817 - accuracy: 0.4188
Epoch 2/400
107/107 [==============================] - 0s 4ms/step - loss: 0.6031 - accuracy: 0.4431
Epoch 3/400
107/107 [==============================] - 0s 3ms/step - loss: 0.5777 - accuracy: 0.4698
Epoch 4/400
107/107 [==============================] - 0s 3ms/step - loss: 0.5626 - accuracy: 0.4936
Epoch 5/400
107/107 [==============================] - 0s 3ms/step - loss: 0.5517 - accuracy: 0.5319
Epoch 6/400
107/107 [==============================] - 1s 6ms/step - loss: 0.5437 - accuracy: 0.5569
Epoch 7/400
107/107 [==============================] - 1s 7ms/step - loss: 0.5376 - accuracy: 0.5689
Epoch 8/400
107/107 [==============================] - 0s 3ms/step - loss: 0.5327 - accuracy: 0.5739
Epoch 9/400
107/107 [==============================] - 0s 3ms/step - loss: 0.5288 - accuracy: 0.5809
Epoch 10/400
107/107 [==============================] - 0s 4ms/step - loss: 0.5252 - accuracy: 0.5835
E

107/107 [==============================] - 0s 3ms/step - loss: 0.4637 - accuracy: 0.6668
Epoch 82/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4633 - accuracy: 0.6621
Epoch 83/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4632 - accuracy: 0.6668
Epoch 84/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4628 - accuracy: 0.6653
Epoch 85/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4629 - accuracy: 0.6615
Epoch 86/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4625 - accuracy: 0.6647
Epoch 87/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4625 - accuracy: 0.6635
Epoch 88/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4622 - accuracy: 0.6641
Epoch 89/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4621 - accuracy: 0.6630
Epoch 90/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4616 - accuracy: 

107/107 [==============================] - 0s 3ms/step - loss: 0.4513 - accuracy: 0.6653
Epoch 161/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4514 - accuracy: 0.6700
Epoch 162/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4511 - accuracy: 0.6694
Epoch 163/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4509 - accuracy: 0.6671
Epoch 164/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4508 - accuracy: 0.6682
Epoch 165/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4505 - accuracy: 0.6671
Epoch 166/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4504 - accuracy: 0.6665
Epoch 167/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4507 - accuracy: 0.6691
Epoch 168/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4500 - accuracy: 0.6706
Epoch 169/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4497 - a

107/107 [==============================] - 0s 3ms/step - loss: 0.4452 - accuracy: 0.6650
Epoch 240/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4458 - accuracy: 0.6659
Epoch 241/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4453 - accuracy: 0.6659
Epoch 242/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4453 - accuracy: 0.6697
Epoch 243/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4453 - accuracy: 0.6653
Epoch 244/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4454 - accuracy: 0.6653
Epoch 245/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4453 - accuracy: 0.6668
Epoch 246/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4451 - accuracy: 0.6665
Epoch 247/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4449 - accuracy: 0.6671
Epoch 248/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4450 - a

107/107 [==============================] - 0s 4ms/step - loss: 0.4428 - accuracy: 0.6723
Epoch 319/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4426 - accuracy: 0.6674
Epoch 320/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4428 - accuracy: 0.6709
Epoch 321/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4426 - accuracy: 0.6659
Epoch 322/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4428 - accuracy: 0.6676
Epoch 323/400
107/107 [==============================] - 1s 5ms/step - loss: 0.4427 - accuracy: 0.6691
Epoch 324/400
107/107 [==============================] - 1s 5ms/step - loss: 0.4423 - accuracy: 0.6647
Epoch 325/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4424 - accuracy: 0.6668
Epoch 326/400
107/107 [==============================] - 0s 3ms/step - loss: 0.4422 - accuracy: 0.6679
Epoch 327/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4424 - a

107/107 [==============================] - 1s 6ms/step - loss: 0.4402 - accuracy: 0.6706
Epoch 398/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4406 - accuracy: 0.6735
Epoch 399/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4403 - accuracy: 0.6712
Epoch 400/400
107/107 [==============================] - 0s 4ms/step - loss: 0.4406 - accuracy: 0.6691


In [8]:
nn_model2.evaluate(X_test_scaled,y_test,verbose=2)

36/36 - 0s - loss: 0.4616 - accuracy: 0.6547 - 349ms/epoch - 10ms/step


[0.4616198241710663, 0.6546573042869568]

In [ ]:

nn_model2.predict(X_test_scaled)

In [ ]:
import pickle


In [ ]:
pickle.dump(nn_model2, open('nn_model.pkl','wb')) 